In [41]:
import cudaq
cudaq.set_target("nvidia", option="fp64")
import numpy as np


from quantum_mcp.mappings import JordanWignerFermion
from quantum_mcp.operator_pools import uccsd_pool
from quantum_mcp.preprocess import MolecularHamiltonian

In [42]:
xyz_fpath = 'co2.xyz'
molecular_data = MolecularHamiltonian(xyz_fpath, 'sto3g', norb_cas=4, nele_cas=4)

molecular_data.run_ccsd()
molecular_data.compute_obi_and_tbi()
jwf = JordanWignerFermion(molecular_data)
spin_ham = jwf.get_spin_hamiltonian()

overwrite output file: co2-pyscf.log
[QC_MCP] == Running HF ==
[QC_MCP] Total number of orbitals = 15
[QC_MCP] Total number of electrons = (11, 11)
[QC_MCP] E[HF] = -185.06320569620112
[QC_MCP] == Running R-CCSD ==
[QC_MCP] == Running CASCI ==
[QC_MCP] E[CASCI] = -185.13555713806633
[QC_MCP] E[R-CCSD] = -185.13517690585297
[QC_MCP] Computing the one- and two-electron integrals
[QC_MCP] Using one- and two-body integrals from CASCI in the HF MO basis
[QC_MCP] Generated the restricted molecular spin Hamiltonian


In [43]:
n_electrons = molecular_data.nele_cas
n_qubits = 2 * molecular_data.nele_cas
molecular_data.nqubits = n_qubits
pools = uccsd_pool(n_electrons, 2 * molecular_data.nele_cas)
print('Number of operator pool: ', len(pools))

sign_pool = []
mod_pool = []
for i in range(len(pools)):
    op_i = pools[i]
    temp_op = []
    temp_coef = []
    
    for term in op_i:
        temp_coef.append(term.evaluate_coefficient())
        temp_op.append(term.get_pauli_word(n_qubits))
        
    
    mod_pool.append(temp_op)
    sign_pool.append(temp_coef)
print(mod_pool)
print(sign_pool)

Number of operator pool:  26
[['YZZZXIII', 'XZZZYIII'], ['YZZZZZXI', 'XZZZZZYI'], ['IIYZXIII', 'IIXZYIII'], ['IIYZZZXI', 'IIXZZZYI'], ['IYZZZXII', 'IXZZZYII'], ['IYZZZZZX', 'IXZZZZZY'], ['IIIYZXII', 'IIIXZYII'], ['IIIYZZZX', 'IIIXZZZY'], ['XXIIXYII', 'XXIIYXII', 'XYIIYYII', 'YXIIYYII', 'XYIIXXII', 'YXIIXXII', 'YYIIXYII', 'YYIIYXII'], ['XXIIIXYI', 'XXIIIYXI', 'XYIIIYYI', 'YXIIIYYI', 'XYIIIXXI', 'YXIIIXXI', 'YYIIIXYI', 'YYIIIYXI'], ['XXIIXZZY', 'XXIIYZZX', 'XYIIYZZY', 'YXIIYZZY', 'XYIIXZZX', 'YXIIXZZX', 'YYIIXZZY', 'YYIIYZZX'], ['XXIIIIXY', 'XXIIIIYX', 'XYIIIIYY', 'YXIIIIYY', 'XYIIIIXX', 'YXIIIIXX', 'YYIIIIXY', 'YYIIIIYX'], ['XZZXXYII', 'XZZXYXII', 'XZZYYYII', 'YZZXYYII', 'XZZYXXII', 'YZZXXXII', 'YZZYXYII', 'YZZYYXII'], ['XZZXIXYI', 'XZZXIYXI', 'XZZYIYYI', 'YZZXIYYI', 'XZZYIXXI', 'YZZXIXXI', 'YZZYIXYI', 'YZZYIYXI'], ['XZZXXZZY', 'XZZXYZZX', 'XZZYYZZY', 'YZZXYZZY', 'XZZYXZZX', 'YZZXXZZX', 'YZZYXZZY', 'YZZYYZZX'], ['XZZXIIXY', 'XZZXIIYX', 'XZZYIIYY', 'YZZXIIYY', 'XZZYIIXX', 'YZZXIIXX', 'YZ

In [44]:
def commutator(pools, ham):
    com_op = []
    
    for i in range(len(pools)):
        # We add the imaginary number that we excluded when generating the operator pool.
        op = 1j * pools[i]
        
        com_op.append(ham * op - op * ham)
         
    return com_op
        
grad_op = commutator(pools, spin_ham)
print('Number of op for gradient: ', len(grad_op))

#for op in grad_op:
#    print(op)
    

Number of op for gradient:  26


In [45]:
# Get the initial state (reference state). 
import cudaq
@cudaq.kernel
def initial_state(n_qubits:int, nelectrons:int):
    
    qubits = cudaq.qvector(n_qubits)
    
    for i in range(nelectrons):
        x(qubits[i])

state = cudaq.get_state(initial_state, n_qubits, n_electrons)
print(state)

SV: [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (1,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (

In [46]:
###################################
# Quantum kernels

@cudaq.kernel
def gradient(state:cudaq.State):
    q = cudaq.qvector(state)


@cudaq.kernel
def kernel(theta: list[float], qubits_num: int, nelectrons: int, pool_single: list[cudaq.pauli_word], 
           coef_single: list[float], pool_double: list[cudaq.pauli_word], coef_double: list[float]):
    q = cudaq.qvector(qubits_num)
    
    for i in range(nelectrons):
        x(q[i])
    
    count=0
    for  i in range(0, len(coef_single), 2):
        exp_pauli(coef_single[i] * theta[count], q, pool_single[i])
        exp_pauli(coef_single[i+1] * theta[count], q, pool_single[i+1])
        count+=1

    for i in range(0, len(coef_double), 8):
        exp_pauli(coef_double[i] * theta[count], q, pool_double[i])
        exp_pauli(coef_double[i+1] * theta[count], q, pool_double[i+1])
        exp_pauli(coef_double[i+2] * theta[count], q, pool_double[i+2])
        exp_pauli(coef_double[i+3] * theta[count], q, pool_double[i+3])
        exp_pauli(coef_double[i+4] * theta[count], q, pool_double[i+4])
        exp_pauli(coef_double[i+5] * theta[count], q, pool_double[i+5])
        exp_pauli(coef_double[i+6] * theta[count], q, pool_double[i+6])
        exp_pauli(coef_double[i+7] * theta[count], q, pool_double[i+7])
        count+=1

In [47]:
from scipy.optimize import minimize

print('Beginning of ADAPT-VQE')

threshold=1e-3
E_prev=0.0
e_stop=1e-5
init_theta=0.0

theta_single=[]
theta_double=[]

pool_single=[]
pool_double=[]

coef_single=[]
coef_double=[]

selected_pool=[]

for i in range(100):
    
    print('Step: ', i)
    
    gradient_vec=[]
    
    for op in grad_op:
        grad=cudaq.observe(gradient, op, state).expectation()
        gradient_vec.append(grad)
    
    norm=np.linalg.norm(np.array(gradient_vec))
    print('Norm of the gradient: ', norm)
    
    
    # When using mpi to parallelize gradient calculation: uncomment the following lines
    
    #chunks=np.array_split(np.array(grad_op), cudaq.mpi.num_ranks())
    #my_rank_op=chunks[cudaq.mpi.rank()]

    #print('We have', len(grad_op), 'pool operators which we would like to split', flush=True)
    #print('We have', len(my_rank_op), 'pool operators on this rank', cudaq.mpi.rank(), flush=True)

    #gradient_vec_async=[]
    
    #for op in my_rank_op:
        #gradient_vec_async.append(cudaq.observe_async(gradient, op, state))

    #gradient_vec_rank=[]
    #for i in range(len(gradient_vec_async)):
    #    get_result=gradient_vec_async[i].get()
    #    get_expectation=get_result.expectation()
    #    gradient_vec_rank.append(get_expectation)
    
    #print('My rank has', len(gradient_vec_rank), 'gradients', flush=True)

    #gradient_vec=cudaq.mpi.all_gather(len(gradient_vec_rank)*cudaq.mpi.num_ranks(), gradient_vec_rank)
    
    
    if norm <= threshold:
        print('\n', 'Final Result: ', '\n')
        print('Final parameters: ', theta)
        print('Selected pools: ', selected_pool)
        print('Number of pools: ', len(selected_pool))
        print('Final energy: ', result_vqe.fun)
        
        break
    
    else:
        
        max_grad=np.max(np.abs(gradient_vec))
        print('max_grad: ', max_grad)
        
        temp_pool = []
        temp_sign = []
        for i in range(len(mod_pool)):
            if np.abs(gradient_vec[i]) == max_grad:
                temp_pool.append(mod_pool[i])
                temp_sign.append(sign_pool[i])
        
        print('Selected pool at current step: ', temp_pool)
        
        selected_pool=selected_pool+temp_pool
        
        tot_single=0
        tot_double=0
        for p in temp_pool:
            if len(p) == 2:
                tot_single += 1
                for word in p:
                    pool_single.append(word)
            else:
                tot_double += 1
                for word in p:
                    pool_double.append(word)
                    
        for coef in temp_sign:
            if len(coef) == 2:
                for value in coef:
                    coef_single.append(value.real)
            else:
                for value in coef:
                    coef_double.append(value.real)
                    
        print('pool single: ', pool_single)
        print('coef_single: ', coef_single)
        print('pool_double: ', pool_double)
        print('coef_double: ', coef_double)
        print('tot_single: ', tot_single)
        print('tot_double: ', tot_double)
        
        init_theta_single = [init_theta] * tot_single
        init_theta_double = [init_theta] * tot_double
        
        theta_single = theta_single + init_theta_single
        theta_double = theta_double + init_theta_double
        print('theta_single', theta_single)
        print('theta_double: ', theta_double)
        
        theta = theta_single + theta_double
        print('theta', theta)
        
        def cost(theta):
            
            theta=theta.tolist()
            
            energy=cudaq.observe(kernel, spin_ham, theta, n_qubits, n_electrons, pool_single, 
                                coef_single, pool_double, coef_double).expectation()
            
            return energy
        
        def parameter_shift(theta):
            parameter_count = len(theta)
            grad = np.zeros(parameter_count)
            theta2 = theta.copy()
            for i in range(parameter_count):
                theta2[i] = theta[i] + np.pi/4
                exp_val_plus = cost(theta2)
                theta2[i] = theta[i] - np.pi/4
                exp_val_minus = cost(theta2)
                grad[i] = (exp_val_plus - exp_val_minus)
                theta2[i] = theta[i]
            return grad
        
        result_vqe=minimize(cost, theta, method='L-BFGS-B', jac='3-point', tol=1e-7)
        # If want to use parameter shift to compute gradient, please uncomment the following line.
        #result_vqe=minimize(cost, theta, method='L-BFGS-B', jac=parameter_shift, tol=1e-7)
        
        theta=result_vqe.x.tolist()
        theta_single = theta[:tot_single]
        theta_double = theta[tot_single:]
        
        print('Optmized Energy: ', result_vqe.fun)
        print('Optimizer exited successfully: ',result_vqe.success, flush=True)
        print(result_vqe.message, flush=True)
        
        dE= result_vqe.fun-E_prev
        print('dE: ', dE)
        print('\n')
        
        if np.abs(dE)<=e_stop:
            print('\n', 'Final Result: ', '\n')
            print('Final parameters: ', theta)
            print('Selected pools: ', selected_pool)
            print('Number of pools: ', len(selected_pool))
            print('Final energy: ', result_vqe.fun)
            
            break
        
        else:
            E_prev=result_vqe.fun
            
            # Prepare a trial state with the current ansatz.
            state=cudaq.get_state(kernel, theta, n_qubits, n_electrons, pool_single, 
                            coef_single, pool_double, coef_double)
            
# When using mpi
#cudaq.mpi.finalize()

Beginning of ADAPT-VQE
Step:  0
Norm of the gradient:  0.6429276587531595
max_grad:  0.2958262282636786
Selected pool at current step:  [['IIXXIIXY', 'IIXXIIYX', 'IIXYIIYY', 'IIYXIIYY', 'IIXYIIXX', 'IIYXIIXX', 'IIYYIIXY', 'IIYYIIYX']]
pool single:  []
coef_single:  []
pool_double:  ['IIXXIIXY', 'IIXXIIYX', 'IIXYIIYY', 'IIYXIIYY', 'IIXYIIXX', 'IIYXIIXX', 'IIYYIIXY', 'IIYYIIYX']
coef_double:  [0.125, 0.125, 0.125, 0.125, -0.125, -0.125, -0.125, -0.125]
tot_single:  0
tot_double:  1
theta_single []
theta_double:  [0.0]
theta [0.0]
Optmized Energy:  -185.08596593422928
Optimizer exited successfully:  True
CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
dE:  -185.08596593422928


Step:  1
Norm of the gradient:  0.5025106399433772
max_grad:  0.2958262282636779
Selected pool at current step:  [['XXIIXYII', 'XXIIYXII', 'XYIIYYII', 'YXIIYYII', 'XYIIXXII', 'YXIIXXII', 'YYIIXYII', 'YYIIYXII']]
pool single:  []
coef_single:  []
pool_double:  ['IIXXIIXY', 'IIXXIIYX', 'IIXYIIYY', 'IIYXIIYY', 'I